In [10]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [11]:
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

[Errno 2] No such file or directory: 'gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks'
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np

class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.b1 = nn.BatchNorm2d(32, momentum=0.15)     
        self.pool = nn.MaxPool2d(2,2)
        self.conv5 = nn.Conv2d(32, 32, 5, padding=2)
        self.conv5_dropout = nn.Dropout(p=0.3)

        self.conv6 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv7 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv8 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv9 = nn.Conv2d(64, 64, 3, padding=1)
        self.b2 = nn.BatchNorm2d(64, momentum=0.15)
        # self.pool = nn.MaxPool2d(2,2) same as upper pool
        self.conv10 = nn.Conv2d(64, 64, 5, padding=2)
        self.b3 = nn.BatchNorm2d(64, momentum=0.15)
        self.conv10_dropout = nn.Dropout(p=0.3)

        self.conv11 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv12 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv13 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv14 = nn.Conv2d(128, 128, 3, padding=1)
        self.b4 = nn.BatchNorm2d(128, momentum=0.15)
        # self.pool = nn.MaxPool2d(2,2) same as upper pool
        self.conv15 = nn.Conv2d(128, 128, 5, padding=2)
        self.b5 = nn.BatchNorm2d(128, momentum=0.15)
        self.conv15_dropout = nn.Dropout(p=0.3)

        self.conv16 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv17 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv18 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv19 = nn.Conv2d(256, 256, 3, padding=1)
        self.b6 = nn.BatchNorm2d(256, momentum=0.15)
        # self.pool = nn.MaxPool2d(2,2) same as upper pool
        self.conv20 = nn.Conv2d(256, 256, 5, padding=2)
        self.b7 = nn.BatchNorm2d(256, momentum=0.15)
        self.conv20_dropout = nn.Dropout(p=0.3)

        self.dense1 = nn.Linear(256*4*4, 1024)
        self.dense1_dropout = nn.Dropout(p=0.3) # +relu
        self.dense2 = nn.Linear(1024, 512) # +relu

        self.head_root = nn.Linear(512, 168) # + softmax
        self.head_vowel = nn.Linear(512, 11) # + softmax
        self.head_consonant = nn.Linear(512, 7) # + softmax





    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.b1(self.conv4(x))))
        x = F.relu(self.conv5(x))
        x = self.conv5_dropout(x)
        
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = self.pool(F.relu(self.b2(self.conv9(x))))
        x = F.relu(self.b3(self.conv10(x)))
        x = self.conv10_dropout(x)

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.pool(F.relu(self.b4(self.conv14(x))))
        x = F.relu(self.b5(self.conv15(x)))
        x = self.conv15_dropout(x)

        x = F.relu(self.conv16(x))
        x = F.relu(self.conv17(x))
        x = F.relu(self.conv18(x))
        x = self.pool(F.relu(self.b6(self.conv19(x))))
        x = F.relu(self.b7(self.conv20(x)))
        x = self.conv20_dropout(x)

        x = x.view(-1, 256*4*4)
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        
        head_root = self.head_root(x)
        head_vowel = self.head_vowel(x)
        head_consonant = self.head_consonant(x)

        return head_root, head_vowel, head_consonant # not sure..

model = model()

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

summary(model, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             320
            Conv2d-2           [-1, 32, 64, 64]           9,248
            Conv2d-3           [-1, 32, 64, 64]           9,248
            Conv2d-4           [-1, 32, 64, 64]           9,248
       BatchNorm2d-5           [-1, 32, 64, 64]              64
         MaxPool2d-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 32, 32, 32]          25,632
           Dropout-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          18,496
           Conv2d-10           [-1, 64, 32, 32]          36,928
           Conv2d-11           [-1, 64, 32, 32]          36,928
           Conv2d-12           [-1, 64, 32, 32]          36,928
      BatchNorm2d-13           [-1, 64, 32, 32]             128
        MaxPool2d-14           [-1, 64,

In [0]:
# このセルはあとで変える
def train(model, epoch):
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print("epoch{} {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss.data))
 
 def test(model):
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        images, labels = data
        outputs = model(Variable(images))
        predicted = torch.max(outputs.data,1)[1]
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print("Accuracy {}/{} {:.2f}%".format(correct, total, 100.0*correct/total))
    
for i in range(1,10):
    train(model, i)
    test(model)
